In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
import deepmatcher as dm
import torch.nn.functional as F
import pandas as pd
import os
from utils.deepmatcher_utils import wrapDm
from deepmatcher.data import MatchingIterator
from utils.intermediate_layer_extraction import return_layer_input

In [5]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('models/walmartamazon_hybrid.pth')
model = model.to('cuda')

In [5]:
def _flat_list(l):
    flat_list = []
    for sublist in l:
        for item in sublist:
            flat_list.append((item))
    return flat_list

In [6]:
def _return_input(module,module_input,module_output):
    global current_layer_input
    if isinstance(module_input,tuple):
        current_layer_input = (module_input[0].detach().requires_grad_())
    else:
        current_layer_input = (module_input[0].detach().requires_grad_())

In [13]:
def return_layer_input_test(model,layer,dataset_dir,dataset_name,batch_size=32,
                       device ='cuda',ignore_columns=['label']):
    dataset = dm.data.process_unlabeled(os.sep.join([dataset_dir,dataset_name+'.csv']),model,
                                        ignore_columns=ignore_columns)
    original_predictions = model.run_prediction(dataset)
    run_iter = MatchingIterator(
            dataset,
            model.meta,
            train=False,
            batch_size=batch_size,
            device=device,
            sort_in_buckets=None)
    tupleids = []
    layer_inputs = []
    hook = layer.register_forward_hook(_return_input)
    for batch_idx,batch in enumerate(run_iter):
        tupleids.append(batch.id)
        model(batch)
        layer_inputs.append(current_layer_input)
    hook.remove()
    for idx,batch in enumerate(layer_inputs):
        batch_pred = model.classifier(batch)
        for j,pred in enumerate(batch_pred):
            match_score = float(pred[1].exp())
            originalPred = original_predictions.loc[tupleids[idx][j]]['match_score']
        if abs(match_score-originalPred)>=0.01:
            print('error')
    return

In [49]:
return_layer_input_test(model,model.classifier,'datasets/Structured/Walmart-Amazon/','negatives')


Reading and processing data from "datasets/Structured/Walmart-Amazon//negatives.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 13


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Finished Epoch 13 || Run Time:   22.2 | Load Time:   14.4 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [ ]:
return_layer_input_test2(model,model.classifier,'datasets/Structured/Walmart-Amazon/','negatives')

[nltk_data] Error loading perluniprops: <urlopen error [Errno -2] Name
[nltk_data]     or service not known>
[nltk_data] Error loading nonbreaking_prefixes: <urlopen error [Errno
[nltk_data]     -2] Name or service not known>
[nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


===>  PREDICT Epoch 13


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:37


Finished Epoch 13 || Run Time:   22.9 | Load Time:   14.5 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 248.38



In [9]:
model1 = dm.MatchingModel(attr_summarizer='hybrid')
model1.load_state('models/itunes_amazon_hybrid.pth')
model1 = model1.to('cuda')

In [15]:
return_layer_input_test(model1,model1.classifier,'datasets/Structured/itunes-amazon','positives')


Reading and processing data from "datasets/Structured/itunes-amazon/positives.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 6


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 6 || Run Time:    0.9 | Load Time:    0.8 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [13]:
def return_layer_input_test2(model,layer,dataset_dir,dataset_name,batch_size=32,
                       device ='cuda',ignore_columns=['label']):
    dataset = dm.data.process(dataset_dir,dataset_name+'.csv',cache=dataset_name+'.pth',
                      left_prefix='ltable_',right_prefix='rtable_')
    original_predictions = model.run_prediction(dataset)
    run_iter = MatchingIterator(
            dataset,
            model.meta,
            train=False,
            batch_size=batch_size,
            device=device,
            sort_in_buckets=None)
    tupleids = []
    layer_inputs = []
    hook = layer.register_forward_hook(_return_input)
    for batch_idx,batch in enumerate(run_iter):
        tupleids.append(batch.id)
        model(batch)
        layer_inputs.append(current_layer_input)
    hook.remove()
    for idx,batch in enumerate(layer_inputs):
        batch_pred = model.classifier(batch)
        for j,pred in enumerate(batch_pred):
            match_score = float(pred[1].exp())
            originalPred = original_predictions.loc[tupleids[idx][j]]['match_score']
        if abs(match_score-originalPred)>=0.01:
            print('error')
    return

In [15]:
return_layer_input_test2(model1,model1.classifier,'datasets/Structured/itunes-amazon/',
                        'test_negatives')

[nltk_data] Error loading perluniprops: <urlopen error [Errno -2] Name
[nltk_data]     or service not known>
[nltk_data] Error loading nonbreaking_prefixes: <urlopen error [Errno
[nltk_data]     -2] Name or service not known>
[nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>

Reading and processing data from "datasets/Structured/itunes-amazon/test_negatives.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


===>  PREDICT Epoch 6
Finished Epoch 6 || Run Time:    0.5 | Load Time:    0.5 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:  80.83

error
error
error


## Test data processing

In [12]:
from sklearn.metrics import f1_score

In [6]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('models/walmartamazon_hybrid.pth')
model = model.to('cuda')

In [10]:
train = dm.data.process_unlabeled('datasets/Structured/Walmart-Amazon/merged_train.csv',model,
                                 ignore_columns=['label'])


Reading and processing data from "datasets/Structured/Walmart-Amazon/merged_train.csv"
0% [############################# ] 100% | ETA: 00:00:00

In [22]:
train_df = pd.read_csv('datasets/Structured/Walmart-Amazon/merged_train.csv')
labels = train_df.label.values
pred = model.run_prediction(train)['match_score'].values

===>  PREDICT Epoch 13


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:24


Finished Epoch 13 || Run Time:   15.1 | Load Time:    9.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [24]:
f1_score(labels,list(map(lambda p:round(p),pred)))

0.9879931389365352

In [17]:
validation = dm.data.process_unlabeled('datasets/Structured/Walmart-Amazon/merged_vali.csv',model,
                                 ignore_columns=['label'])
validation_df = pd.read_csv('datasets/Structured/Walmart-Amazon/merged_valid.csv')
labels = validation.label.values
pred = model.run_prediction(train)['match_score'].values

===>  EVAL Epoch 13


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 13 || Run Time:    5.1 | Load Time:    3.4 || F1:  64.23 | Prec:  60.55 | Rec:  68.39 || Ex/s: 240.22



tensor(64.2336, device='cuda:0')

In [15]:
model.run_eval(neg3)

===>  EVAL Epoch 13


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 13 || Run Time:    5.2 | Load Time:    3.5 || F1:  61.58 | Prec:  62.57 | Rec:  60.62 || Ex/s: 236.90



tensor(61.5789, device='cuda:0')